In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import dask.dataframe as dd

trade_df = dd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet')
trade_df.head()

In [ ]:
f'{len(trade_df):,} rows'

In [ ]:
book_df = dd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet')
book_df.head()

In [ ]:
f'{len(book_df):,} rows'

In [ ]:
f'{len(set(book_df.stock_id))} stocks'

In [ ]:
trade_df.price.describe().compute()

In [ ]:
import plotly.express as px

px.line(trade_df.groupby('seconds_in_bucket')['price'].std().compute())

In [ ]:
time_ids = pd.Series(list(set(trade_df.time_id)))
print(f'{len(time_ids)} time ids')

time_ids

In [ ]:
random_time = time_ids.sample(1).values[0]
df = trade_df[trade_df.time_id == random_time].compute()

In [ ]:
df.__class__

In [ ]:
def f(partition):
    return partition.name

trade_df.groupby('time_id').apply(f, meta=object).compute()

In [ ]:
%%writefile same.py
import pandas as pd
import numpy as np

train_labels = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/train.csv')

def eval_strat(df):
    price_df = df.set_index(['seconds_in_bucket', 'stock_id'])['price'].unstack().fillna(
            method='pad')
    
    pred_rv = (price_df.apply(np.log).diff()**2).sum().apply(np.sqrt).rename('pred_rv').reset_index()
    df = train_labels[train_labels.time_id==df.name].merge(pred_rv)
    
    return (((df.target-df.pred_rv)/df.target)**2).mean()**0.5

In [ ]:
import same
import importlib

importlib.reload(same)
df = trade_df[trade_df.time_id == random_time].compute()
df.name = random_time
same.eval_strat(df)

In [ ]:
evals = trade_df.groupby('time_id').apply(same.eval_strat, meta=object).compute()
evals

In [ ]:
evals.describe()